# Applied Data Science Capstone Assignment 2 :Segmenting and Clustering Neighborhoods in Toronto


#### For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

Start by creating a new Notebook for this assignment.
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

In [1]:
# Import necessary libraries

import requests
import lxml.html as lh
import pandas as pd

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [3]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

#### This means that there are 3 columns per row

In [4]:
# Parse the first row as our header
tr_elements = doc.xpath('//tr')

#Create empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postal Code
"
2:"Borough
"
3:"Neighborhood
"


#### Creating Pandas DataFrame
### Each header is appended to a tuple along with an empty list.

In [5]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [6]:
# Check the length of each column. Ideally, they should all be the same
[len(C) for (title,C) in col]

[181, 181, 181]

##### This shows that each of the 3 columns has exactly 181 rows

### Creating the pandas data frame

In [7]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

3. To create the above dataframe:

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

4. Submit a link to your Notebook on your Github repository. (10 marks)

Note: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas to read the table into a pandas dataframe.

Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.

In [8]:
# Access the top 5 rows of the data frame 
df.head()

,Postal Code,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [9]:
df = df.replace('\n',' ', regex=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Dropping all cells with a borough that is Not assigned

In [10]:
df.rename(columns = {'Postal Code\n':'Postal Code','Borough\n':'Borough','Neighborhood\n':'Neighbourhood'}, inplace = True) 

In [11]:
df.columns

Index(['Postal Code', 'Borough', 'Neighbourhood'], dtype='object')

In [12]:
df['Borough'].unique

<bound method Series.unique of 0               Not assigned 
1               Not assigned 
2                 North York 
3                 North York 
4           Downtown Toronto 
5                 North York 
6           Downtown Toronto 
7               Not assigned 
8                  Etobicoke 
9                Scarborough 
10              Not assigned 
11                North York 
12                 East York 
13          Downtown Toronto 
14                North York 
15              Not assigned 
16              Not assigned 
17                 Etobicoke 
18               Scarborough 
19              Not assigned 
20                North York 
21                 East York 
22          Downtown Toronto 
23                      York 
24              Not assigned 
25              Not assigned 
26                 Etobicoke 
27               Scarborough 
28              Not assigned 
29              Not assigned 
                ...          
151                Etobicoke 
152      

### Dropping all cells with a borough that is Not assigned


In [13]:
df.drop(df.index[df["Borough"] == 'Not assigned '], inplace = True)

# Reset the index and dropping the previous index
df = df.reset_index(drop=True)

df.head(10)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [14]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [15]:
df['Neighbourhood'].unique()

array(['Parkwoods ', 'Victoria Village ', 'Regent Park, Harbourfront ',
       'Lawrence Manor, Lawrence Heights ',
       "Queen's Park, Ontario Provincial Government ",
       'Islington Avenue, Humber Valley Village ', 'Malvern, Rouge ',
       'Don Mills ', 'Parkview Hill, Woodbine Gardens ',
       'Garden District, Ryerson ', 'Glencairn ',
       'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale ',
       'Rouge Hill, Port Union, Highland Creek ', 'Woodbine Heights ',
       'St. James Town ', 'Humewood-Cedarvale ',
       'Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood ',
       'Guildwood, Morningside, West Hill ', 'The Beaches ',
       'Berczy Park ', 'Caledonia-Fairbanks ', 'Woburn ', 'Leaside ',
       'Central Bay Street ', 'Christie ', 'Cedarbrae ',
       'Hillcrest Village ',
       'Bathurst Manor, Wilson Heights, Downsview North ',
       'Thorncliffe Park ', 'Richmond, Adelaide, King ',
       'Dufferin, Dovercourt Village ', 'Sca

#### Combining Neighbourhoods based on similar Postcode and Borough

In [16]:
df = df.groupby(['Postal Code', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postal Code','Borough','Neighbourhood']
df.head(10)

,Postal Code,Borough,Neighbourhood
0,,Canadian postal codes,
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae
6,M1J,Scarborough,Scarborough Village
7,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
8,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
9,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"


#### Removing any space in the start of the string

In [17]:
df['Neighbourhood'] = df['Neighbourhood'].str.strip()

#### Assigning Borough values to the Neignbourhood where vlaue is "Not assigned"

In [18]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

In [19]:
# Check if the Neighbourhood for Queen's Park changed 
df[df['Borough'] == 'Queen\'s Park']

,Postal Code,Borough,Neighbourhood


In [20]:
df.head()

,Postal Code,Borough,Neighbourhood
0,,Canadian postal codes,
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn


In [21]:
# Check the shape of the data frame
df.shape

(104, 3)

In [22]:
df = df[1:]

In [23]:
df.shape

(103, 3)